In [1]:
import pandas as pd, pickle
import numpy as np
from pmdarima.arima import AutoARIMA
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import date as dt, timedelta, datetime
import yfinance as yf
import pymysql
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import pandas_datareader as web
from dateutil.relativedelta import *
import yfinance as yf
from yahoofinancials import YahooFinancials
from sqlalchemy import create_engine
import mysql.connector
from pandas.tseries.offsets import BDay

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from feature_engine.selection import DropCorrelatedFeatures, SmartCorrelatedSelection

In [2]:
#We got the combined dataset from another file
df = pd.read_excel('test_8.4.xlsx', sheet_name='Sheet1')
df.shape

(183, 124)

In [3]:
def remove_null(df, fraction=0.7):
    
#     df = pd.read_excel('test_8.4.xlsx')
    
    num_rows = len(df)
    num_cols = df.shape[1]
    
    #Deleting the row that has 20 missing values of the complete columns
    df.dropna(axis=0, thresh=len(df.columns)-5, inplace=True)
    
    #Deleting columns that has atleast certain % of missing value
    df.dropna(axis=1, thresh=fraction, inplace=True)
    
    #Drop columns & rows with all null values 
    df.dropna(how='all', inplace=True, axis=1)
    df.dropna(how='all', inplace=True, axis=0)
    
    return df, num_rows, num_cols

df, num_rows, num_cols = remove_null(df)

In [4]:
def clean_impute(df1):
    
    df, num_rows, num_cols = remove_null(df1)
    
    df.drop(columns=['perc_change', 'time_tod', 'Latest Earnings', 'Price/Cash Flow'], inplace=True)

    cat_list = df.select_dtypes('O').columns.to_list()
    num_list = list(set(df.columns.to_list()).difference(set(cat_list)))
    print(f'B4 transformation length of cat: {len(cat_list)}, len of num: {len(num_list)}')
          
    #for categorical variable, cleaning ones with % & , and converting them to float
    df.replace(['unch'], 0, inplace=True)
    for col in df.select_dtypes('O').columns.to_list():

        df[col] = df[col].str.replace(r'%', "")
        df[col] = df[col].str.replace(r',', "")

        try:
            df[col] = df[col].astype('float')
        except:
            pass
        
    df['Trend Str'].fillna('Average', inplace=True)
    
    #for numeric variables
    df['% Institutional'].fillna(50, inplace=True)
    df['% Insider'].fillna(10, inplace=True)
    df['EPS Growth Prv(q)'].fillna(0, inplace=True)
    
    for col in df.columns:
        if col.endswith('Qtrs Ago'):
            df[col].fillna(0, inplace=True)
    
    df.dropna(axis=0, inplace = True)
    
    new_rows, new_cols = df.shape[0], df.shape[1]
    num_rows_dropped, num_cols_dropped = num_rows - new_rows, num_cols - new_cols
    
    cat_list = df.select_dtypes('O').columns.to_list()
    num_list = list(set(df.columns.to_list()).difference(set(cat_list)))
    
    print(f'After transformation length of cat: {len(cat_list)}, len of num: {len(num_list)}')
    
    print(f'number of rows dropped: {num_rows_dropped}')
    print(f'number of cols dropped: {num_cols_dropped}')
    
    
    return df

In [5]:
df = clean_impute(df)

B4 transformation length of cat: 64, len of num: 56
After transformation length of cat: 15, len of num: 105
number of rows dropped: 13
number of cols dropped: 4


In [6]:
def analyst_col_clean(df):
    
    #Shorten this code when you have time

    ap_list, lw_list, mt_list = [], [], []
    
    op_list = list(df['Opinion'].values)
    for item in op_list:
        try:
            new = item.split(" ")[1]
            value = item.split(" ")[0]

            if new == 'Buy':
                value = int(value)*1
            elif new == 'Sell':
                value = int(value)* -1

        except:
            value = 0 


        ap_list.append(value)


    lw_list_o = list(df['Last Week'].values)
    for item in lw_list_o:
        try:
            new = item.split(" ")[1]
            value = item.split(" ")[0]

            if new == 'Buy':
                value = int(value)*1
            elif new == 'Sell':
                value = int(value)* -1

        except:
            value = 0 


        lw_list.append(value)

    mt_list_o = list(df['Medium Term'].values)
    for item in mt_list_o:
        try:
            new = item.split(" ")[1]
            value = item.split(" ")[0]

            if new == 'Buy':
                value = int(value)*1
            elif new == 'Sell':
                value = int(value)* -1

        except:
            value = 0 


        mt_list.append(value)
        
    df['Opinion'], df['Last Week'], df['Medium Term'] = ap_list, lw_list, mt_list
    opinion_cols = ['Opinion', 'Last Week', 'Medium Term']
    
    for col in opinion_cols:
        df[col] = df[col].astype('float')
    
    return df

In [7]:
new_df = analyst_col_clean(df)

In [8]:
new_df.head()

,Symbol,Analyst Rating,# Analysts,Earnings Est,Earnings Est 2-Qtrs Ago,Earnings Est 3-Qtrs Ago,Earnings Est 4-Qtrs Ago,Reported,Reported 2-Qtrs Ago,Reported 3-Qtrs Ago,...,20D MA,20D MA Str,50D MA,50D MA Str,20-50D MACD,Opinion,Prev Signal,Last Week,Medium Term,Target
0,ACLS,4.8,5,0.92,0.84,0.71,0.45,1.22,1.05,0.81,...,Buy,Strong,Buy,Maximum,Buy,24.0,Buy,-8.0,-50.0,1
1,ADTN,4.4,5,0.02,-0.06,0.14,0.12,0.17,0.06,-0.06,...,Buy,Average,Buy,Maximum,Buy,88.0,Buy,56.0,100.0,0
3,ALG,3.0,1,1.69,0.00,1.98,1.56,1.63,0.00,1.59,...,Buy,Soft,Buy,Soft,Buy,-24.0,Sell,-40.0,-50.0,1
4,AMOT,5.0,1,0.09,0.30,0.30,0.26,0.24,0.20,0.41,...,Buy,Maximum,Buy,Strong,Buy,-24.0,Sell,-40.0,-50.0,1
6,ANIK,3.0,2,-0.24,0.00,0.08,0.04,-0.11,-0.23,0.05,...,Buy,Soft,Buy,Soft,Buy,-24.0,Sell,-24.0,-50.0,0


In [9]:
def label_encoder(df):
    
    le = LabelEncoder()
    cate_list = new_df.select_dtypes(include='O').columns.to_list()[1:] # We don't want the symbol
    new_df[cate_list] = new_df[cate_list].apply(lambda x: le.fit_transform(x.astype(str)))
    
    pickle.dump(le, open('label_encoder.pkl', 'wb'))
    
    return df

In [10]:
new_df = label_encoder(new_df)
new_df.shape

(158, 120)

In [11]:
# From feature Engine removing correlated features - this method is much easier than above
fe_corr = DropCorrelatedFeatures(threshold=0.9, method='pearson', missing_values='ignore')
new_df = fe_corr.fit_transform(new_df)

In [12]:
new_df.shape

(158, 82)

In [13]:
fe_corr.correlated_feature_sets_[:2]

[{'Earnings',
  'Earnings Est',
  'Earnings Est 2-Qtrs Ago',
  'Earnings ttm',
  'Reported',
  'Reported 2-Qtrs Ago'},
 {'Earnings Est 3-Qtrs Ago',
  'Earnings Est 4-Qtrs Ago',
  'Reported 3-Qtrs Ago',
  'Reported 4-Qtrs Ago'}]

In [14]:
def train_test(new_df):

# scaler= MinMaxScaler()

    X, y = new_df.iloc[:,1:-1], new_df.iloc[:,-1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True, stratify=y)
    
    return new_df, X_train, X_test, y_train, y_test, X, y

In [15]:
new_df, X_train, X_test, y_train, y_test, X, y =  train_test(new_df)

In [16]:
#Checking if we have any duplicate columns meaning two columns having same value
def getDuplicateColumns(df):
    
    dup_col = {}  
    duplicateColumnNames = set()
    for x in range(df.shape[1]):

        col = df.iloc[:, x]
        column_name = col.name
          
        for y in range(x + 1, df.shape[1]):
            otherCol = df.iloc[:, y]
            other_col_name = otherCol.name
              
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
                dup_col[column_name] = other_col_name
                  
    return dup_col #,list(duplicateColumnNames)

getDuplicateColumns(X_train)

{}

# Removing Constant and Quasi Constant 

In [17]:
def removing_constant(X_train, X_test):

    X_train_constant = [feat for feat in X_train.columns if X_train[feat].nunique() == 1]
    X_test_constant = [feat for feat in X_test.columns if X_test[feat].nunique() == 1]
    print(len(X_train_constant), len(X_test_constant))
    print(f'features with constant values in X_train & X_test {X_train_constant}, {X_test_constant}')

    X_train.drop(columns=X_train_constant, inplace=True)
    X_test.drop(columns=X_test_constant, inplace=True)
    
    #Removing quasi-Constant feature with variance Threshold
    vt = VarianceThreshold(threshold=0.02)
    vt.fit(X_test)
    vt.fit(X_train)
    
    return X_train, X_test

X_train, X_test = removing_constant(X_train, X_test)

1 1
features with constant values in X_train & X_test ['Options'], ['Options']


In [18]:
X_train.shape, X_test.shape

((105, 79), (53, 79))

# Feature Selection by SFP

In [19]:
from sklearn.feature_selection import SequentialFeatureSelector as SFS
sfs = SFS(
    estimator=RandomForestClassifier(
    n_estimators=10, n_jobs=4, random_state=0),
    n_features_to_select=10,  # the number of features to retain
    direction='forward',  # the direction of the selection procedure
    scoring='roc_auc',  # the metric to evaluate
    cv=2,  # the cross-validation fold
    n_jobs=4,  # for parallelization
)

sfs = sfs.fit_transform(X_train, y_train)

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score, accuracy_score
DT = DecisionTreeClassifier().fit(X_train,y_train)

params_grid = {'max_depth':range(1, DT.tree_.max_depth+1, 2),
               'max_features':range(1,len(DT.feature_importances_)+1)
              }

DT_GV = GridSearchCV(DecisionTreeClassifier(random_state=101),
                   param_grid=params_grid,
                   scoring='accuracy',
                   n_jobs=-1)

DT_GV = DT_GV.fit(X_train,y_train)
y_predDT = DT_GV.predict(X_test)

In [21]:
print(classification_report(y_test,y_predDT))

              precision    recall  f1-score   support

           0       0.44      0.43      0.44        28
           1       0.38      0.40      0.39        25

    accuracy                           0.42        53
   macro avg       0.41      0.41      0.41        53
weighted avg       0.42      0.42      0.42        53



In [25]:
#Adaboost
parameters = {'learning_rate': [ 0.5, 0.2, 0.1, 0.01, 0.001], 
              #'subsample':[1.0, 0.5,0.2], 
              #'max_features':[4, 5, 10, 12, 19],
              'n_estimators':[15, 20, 30, 40, 80, 100, 200, 400]
             }

AB = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, scoring='accuracy', n_jobs=-1)
AB = AB.fit(X_train,y_train)
y_predAB = AB.predict(X_test)
print(classification_report(y_test,y_predAB))

              precision    recall  f1-score   support

           0       0.39      0.32      0.35        28
           1       0.37      0.44      0.40        25

    accuracy                           0.38        53
   macro avg       0.38      0.38      0.38        53
weighted avg       0.38      0.38      0.38        53

